In [143]:
import pandas as pd

Form a graph with people as nodes with edges between them if they have starred in the same movie. Search for some correlation between how well connected people are and the kind of movies they have performed in / the type of role they have etc.
Once such hypotheses have been formed, test them in a similar graph in the imdb movie dataset to see if they hold there as well. 

Test the hypothesis that an uncommon combination of genres means a somewhat extreme box office collection - either low or high



Is benford's law scale invariant?

In [144]:
movie_df = pd.read_csv('data/movie.metadata.tsv', sep='\t', names=['wiki_movie_id', 'freebase_movie_id', 'title', 'release_date', 'box_office_revenue', 'runtime', 'languages', 'countries', 'genres'], index_col='freebase_movie_id')
movie_df = movie_df.reset_index()

In [145]:
# No duplicates of freebase_movie_id
movie_df.groupby('freebase_movie_id').agg(count = ('freebase_movie_id', 'size')).sort_values('count', ascending=False)

,count
freebase_movie_id,
/m/011_mj,1
/m/0bwg_bz,1
/m/0bwgk61,1
/m/0bwgjgq,1
/m/0bwghsn,1
...,...
/m/04n51dq,1
/m/04n5187,1
/m/04n514t,1


In [ ]:
col_names = ["wiki_movie_id", "freebase_movie_id", "movie_release_date", "character_name", "actor_date_of_birth", "Actor gender", "Actor height (in meters)", "Actor ethnicity (Freebase ID)", "Actor name", "Actor age at movie release", "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"]
character_df = pd.read_csv('data/character.metadata.tsv', sep='\t', names=col_names, index_col="Freebase character ID")

# Remove characters without actors
character_df = character_df[character_df['Freebase actor ID'].notna()]

# Allow only one character per actor per movie.
# Motivation: When we look at if the actor won a Oscar we don't care about what role it won it as. 
character_df = character_df.groupby(['freebase_movie_id', 'Freebase actor ID']).first().reset_index()

# How many cases where the actors of the same name stars in the same movie or
temp = character_df.groupby(['freebase_movie_id', 'Actor name']).agg(count = ('Actor name', 'size')).reset_index()
movies_with_actors_of_same_name_df = temp[temp['count'] > 1]
print('Number of movies with actors of the same name:', movies_with_actors_of_same_name_df.shape[0])

# Only 311 we decide to drop the cases.
# TODO: Check if any of the 311 actors won Oscars
character_df = character_df.merge(temp, on=['freebase_movie_id', 'Actor name'], how='inner')
dropped_character_df = character_df[character_df['count'] == 1]

temp = dropped_character_df.groupby(['freebase_movie_id', 'Actor name']).agg(count = ('Actor name', 'size')).reset_index()

assert temp[temp['count'] > 1].empty

character_df = dropped_character_df

Number of movies with actors of the same name: 311


In [160]:
movie_character_df = movie_df.merge(character_df, on='freebase_movie_id', how='inner')

In [163]:
# To join with Oscar Movie name and Actor name need to uniquely identify one row in movie_character_df 
assert movie_character_df.groupby(['title', 'Actor name']).agg(count = ('Actor name', 'size')).max()['count'] == 1

In [179]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')

# Removes weird Oscar nomination
oscar_df = oscar_df[oscar_df['film'].notna() & oscar_df['name'].notna()]

In [180]:
# Filtering relevant Oscar catagories
# TODO: explain why we drop certain categories.
oscar_df = oscar_df[oscar_df['category'].str.contains('ACTOR') | oscar_df['category'].str.contains('ACTRESS')]

This join needs to be checked. 
Joining with ignore case? 
Do we want to do a right join? (so also keep movies that did not win oscars)


In [181]:
# Avoid case-sensitivity and extra spaces
oscar_df['film_title_parsed'] = oscar_df['film'].str.lower().str.strip()
oscar_df['name_parsed'] = oscar_df['name'].str.lower().str.strip()

movie_character_df['film_title_parsed'] = movie_character_df['title'].str.lower().str.strip()
movie_character_df['name_parsed'] = movie_character_df['Actor name'].str.lower().str.strip()

In [182]:
# One actor won two Oscars for the same movie (Barry Fitzgerald)
oscar_df.groupby(['name_parsed', 'film_title_parsed']).agg(count = ('name_parsed', 'count')).sort_values('count', ascending=False)

# If actor has mutiple nomination for the same movie we choose the first
oscar_df = oscar_df.groupby(['name_parsed', 'film_title_parsed']).first().reset_index()

# For joining rows in oscar_df needs to be uniquely identified by name_parsed and film_title_parsed
assert oscar_df.groupby(['name_parsed', 'film_title_parsed']).agg(count = ('name_parsed', 'size')).max()['count'] == 1

In [183]:
movie_character_oscar_df = movie_character_df.merge(oscar_df, how='outer', on=['film_title_parsed', 'name_parsed'])

In [ ]:
oscar_nominated_actors_df = movie_character_oscar_df[movie_character_oscar_df['film'].notna()]
print('Number of different Oscar nominated films in dataset:', 
      oscar_nominated_actors_df['film_title_parsed'].unique().shape[0])

print('Number of different Oscar nominated actors in dataset:', 
      oscar_nominated_actors_df['name_parsed'].unique().shape[0])

print('Number of characters in dataset:', oscar_nominated_actors_df.shape[0])
print('Number of movies in dataset:', oscar_nominated_actors_df[''].unique().shape[0])
print('Number of actors in dataset:', oscar_nominated_actors_df[''].unique().shape[0])

Number of different Oscar nominated films in dataset: 1201
Number of different Oscar nominated actors in dataset: 982
Number of characters in dataset: 1827
Number of movies in dataset: 1201
